# Creating manual annotations and training Cellpose
*General tip 1*: Whenever you modify anything in this notebook, do not forget to click on the save button. In this way, all changes will be saved and you will be able to continue working on your project later on.

*General tip 2*: To run a part of this notebook, you should first click on a section/cell (after selecting it, you should see a blue bar on the left side) and then click the Play button above (alternatively, press *Shift+Enter*). After running a cell, the next one will be automatically selected.

## 1) Select project directory
Here you have two options:

- If you are *starting a new project*, select an empty folder: all generated data and labels will be saved in this folder.
- If you want to *keep working with an exhisting project*, then select the previous project folder: pre-existing data will be automatically loaded.

To get the path of the selected project directory, you can click on the "Browse..." button below. Then you will be able to copy the displayed path in the code-cell below (see comments below).

In [1]:
%load_ext autoreload
%autoreload 2

import os
import ipywidgets as widgets
from ipywidgets import Button
from tkinter import Tk, filedialog
from IPython.display import clear_output, display
from IPython.display import Markdown, Latex

selected_directory = widgets.Label(value="")


def select_proj_dir(b):
    clear_output()
    root = Tk()
    root.withdraw()  # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True)  # Raise the root to the top of all windows.
    b.directory = filedialog.askdirectory()
    # b.files = filedialog.askopenfilename(multiple=True) # List of selected files will be set button's file attribute.
    selected_directory.value = "Copy this path below: \"{}\"".format(b.directory)
    # with out_proj_dir:        
    #     print("Selected project directory: {}".format(b.directory))


fileselect = Button(description="Browse...")
fileselect.on_click(select_proj_dir)

widgets.VBox(
    [widgets.HBox([widgets.Label(value="Click to select a project directory: "), fileselect]), selected_directory])

In [2]:
# !!! Do not forget to re-run this cell and the following one every time you change any path !!!

# --------------------  TO BE FILLED BY USER - START  -----------------------------

# Insert path to project directory here:
project_directory = "/Users/alberto-mac/EMBL_ATeam/cellpose_training_pipeline/test_project"  # For example: "/home/my_home/data/my_first_project"

# ---------------------  TO BE FILLED BY USER - END  ------------------------------

In [9]:
from annotationtools.base_experiment import BaseAnnotationExperiment
annotation_exp = BaseAnnotationExperiment(project_directory)

input_images_with_nb_rois = annotation_exp.get_list_rois_per_image()
echo = f"""### Name loaded project: '*{os.path.split(project_directory)[1]}*' 
"""
nb_images = len(input_images_with_nb_rois)
if nb_images:
    echo += f"""There are {nb_images} images loaded in the project:
    """
else:
    echo += f"""There are no images loaded in the project.
    """


for i, in_image in enumerate(input_images_with_nb_rois):
    echo += f"""
- Image {i+1}:
    - Path main image: *{in_image[0]}*
    - Number of selected regions of interest in image: *{in_image[1]}*  
    """
    
display(Markdown(echo))

### Name loaded project: '*test_project*' 
There are 1 images loaded in the project:
    
- Image 1:
    - Path main image: */Users/alberto-mac/EMBL_ATeam/cellpose_training_pipeline/test_images/img1/fused_tp_0_ch_4.tif*
    - Number of selected regions of interest in image: *0*  
    

**TODO:** Explain: current crops are listed above 

## 2) Select regions of interest that will be annotated
**Important**: You only need to run section (2) of the notebook if you want to *add* or *modify* regions of interest. If you have already selected all the region of interests you need, then you can move on to section 3.

In this section, you will load an image and then select one or more region of interests (rectangular boxes) that you will manually annotate in the following and will be used to train the segmentation algorithm. 

### Things to keep in mind when selecting regions of interest
- When you select a region of interest, keep in mind that **you will need to manually annotate all cells inside that region of interest**. Each region of interest should contain **at least several dozens of cells, ideally ~100**.
- These region of interests will be used to train the segmentation algorithm, so you should select the most representative parts of your data. If there are some particularly challenging parts of your data the you would like the segmentation algorithm to properly segment, then you should also select some regions of interest from these difficult parts. 

Here you have some example scenarios:
1. Your images/datasets present several artifacts close to the borders, but you only care to have an accurate segmentation in the central part of the image, where image quality is higher and there are no artifacts. In this case, you can select regions of interest only from the central area of your image.
2. Your images/datasets present several artifacts in all parts of the image, and you would like cells to be properly segmented even in parts of the image where these artifacts are present. Then, you should also include regions of interest where some of the artifacts are visible (on top of region of interests that are easier to segment)
3. You acquired several images that look somehow different (different acquisition method, brightness, etc). Then, you should evenly select regions of interest across all your images/datasets.

### 2.1) Load the image
#### Option 1: Add/modify crops from a previously loaded image
From the Drop-list menu below, select the image you want to work with. Then, go to step 2.2.

In [4]:
input_images_with_nb_rois = annotation_exp.get_list_rois_per_image()

dropdown_widget = widgets.Dropdown(
    options=[(img_data[0], i)  for i, img_data in enumerate(input_images_with_nb_rois)],
    value=0,
    disabled=False,
    layout=widgets.Layout(width='90%')
)

id_selected_image = 0

def on_dropdown_value_change(change):
    id_selected_image = change['new']

dropdown_widget.observe(on_dropdown_value_change, names='value')

image_choice_widgets = widgets.VBox([widgets.Label(value="To update the ROIs of an image already in the project, select: "), dropdown_widget])
display(image_choice_widgets)


#### Option 2: Add crops for a new image

In [21]:
selected_file = widgets.Label(value="")


def select_file(b):
    clear_output()
    root = Tk()
    root.withdraw()  # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True)  # Raise the root to the top of all windows.
    b.file = filedialog.askopenfilename()  # List of selected files will be set button's file attribute.
    selected_file.value = "Copy selected path below: \"{}\" ".format(b.file)


fileselect = Button(description="Browse files...")
fileselect.on_click(select_file)

widgets.VBox([widgets.HBox([widgets.Label(value="To get the path of an image, you can click on this button: "), fileselect]), selected_file])

Please fill the code section below with the paths of the following images:

- Main image that should be segmented: this is usually the bright-field or GFP channel
- Optional: image with DAPI channel (if you have it, then add it, because it can be very useful for the segmentation algorithm)

If you have additional channels that may helpful for doing a precise manual annotation, you can load two additional channels... **TODO break code in several blocks...**

In [60]:
# --------------------  TO BE FILLED BY USER - START  -----------------------------

# Main image to be segmented:
main_image_path = "/Users/alberto-mac/EMBL_ATeam/cellpose_training_pipeline/test_images/img1/fused_tp_0_ch_4.tif"

use_dapi_channel_for_segmentation = True

# For additional (optional) channels, you can either specify the image paths...
dapi_image_path = ""
extra_ch_1_image_path = ""
extra_ch_2_image_path = ""

# ...or you can define some name filters, so that additional channels are automatically found in the same
# directory of the main image:
filter_main_image = "_ch_4"
filter_dapi_image = "_ch_2"
filter_extra_ch_1 = "_ch_1"
filter_extra_ch_2 = ""

# If you want, you can also define the extra channel names, for clarity:
extra_ch_1_name = "GFP"
extra_ch_2_name = "Extra channel 2"

# ---------------------  TO BE FILLED BY USER - END  ------------------------------

In [62]:
# Add new image to experiment:

# TODO: add check and only execute if paths were given! (User may fill option 1 and then execute this code!)

annotation_exp.use_dapi_channel_for_segmentation = use_dapi_channel_for_segmentation
annotation_exp.set_extra_channels_names([extra_ch_1_name, extra_ch_2_name])
id_selected_image = annotation_exp.add_input_image(main_image_path,
                               filter_main_image,
                               dapi_image_path,
                               filter_dapi_image,
                               extra_ch_1_image_path,
                               filter_extra_ch_1,
                               extra_ch_2_image_path,
                               filter_extra_ch_2)

The added image was already present in the project. Updating paths.


### 2.2) Modify/create region of interest using Napari
After you run the next cell, the selected image will be loaded in Napari. 

Then select the "Rectangle" tool in napari to draw one or more region of interests.

After you are done, close the Napari window and run the  the next sections 

- Select, move, and delete boxes
- Insert image of Napari viewer? (Showing the various buttons)

![Napari](./napari-screenshot.jpg)

In [30]:
from segmfriends.io.images import read_uint8_img
import napari

assert id_selected_image is not None

image_paths = annotation_exp.get_image_paths(id_selected_image)

# create the viewer and display the image
viewer = napari.Viewer()

for channel in image_paths:
    viewer.add_image(read_uint8_img(image_paths[channel]), name=channel)

napari_rois = annotation_exp.get_napari_roi_by_image_id(id_selected_image)
    
# Load images in napari:
s_layer = viewer.add_shapes(data=napari_rois, name="Crops", shape_type="rectangle", opacity=0.15,  edge_color='#fff01dff', face_color='#f9ffbeff')
# draw some rectangles, then inspect
# s_layer.data

Warning, image fused_tp_0_ch_4.tif has type uint16 and it was converted to uint8
Warning, image fused_tp_0_ch_2.tif has type uint16 and it was converted to uint8
Warning, image fused_tp_0_ch_1.tif has type uint16 and it was converted to uint8


In [27]:
# TODO: add event to napari and call this every time a ROI is updated:
# print(napari_rois.shape)

ROIs_data = s_layer.data
if len(ROIs_data):
    annotation_exp.update_rois_image(id_selected_image, ROIs_data)
else:
    print("No ROIs found for selected image")

# Other stuff

In [11]:


global a1
a = widgets.IntSlider(description='a')
b = widgets.IntSlider(description='b')
c = widgets.IntSlider(description='c')


def f(a, b, c):
    print('{}*{}*{}={}'.format(a, b, c, a * b * c))


out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

widgets.HBox([widgets.VBox([a, b, c]), out])

In [ ]:
img_main = widgets.Text(
    value='',
    placeholder='Enter directory path',
    disabled=False
)

widgets.HBox([widgets.VBox([a, b, c]), out])

In [8]:
print(a.value)

42


In [12]:

caption = widgets.Label(value='The selected .')
slider = widgets.IntSlider(min=-5, max=5, value=1, description='Slider')


def handle_slider_change(change):
    caption.value = 'The slider value is ' + (
        'negative' if change.new < 0 else 'nonnegative'
    )


slider.observe(handle_slider_change, names='value')

display(caption, slider)

Label(value='The selected .')

IntSlider(value=1, description='Slider', max=5, min=-5)

In [13]:
int_range = widgets.IntSlider()
output2 = widgets.Output()

display(int_range, output2)


def on_value_change(change):
    with output2:
        print(change['new'])


int_range.observe(on_value_change, names='value')

IntSlider(value=0)

Output()